In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-26"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10598,2024-03-26,República Tcheca Zbl Feminina,14:00,USK Prague F,Trutnov F,1.63,2.09,143.5,1.85,1.85,0.0,0.0,0.0,U59NvBwA,0.613497,0.478469,0.540541,0.540541,0.091966,552.594,956.299198,1.730564,3.0,0.000000,0.000000,165.98,6.346,10.993590,1.732365,189.0,130.562,89.200505,0.683204,1.2,1.643168,1.369306,60.90,2.030,0.742395,0.365712,-83.0,86,35,1.93,1.74,116.168,261.794,0.174876,0.000000,NaN,1.55,0.310,2.032258,0.000000,0.0,0.000000,-3.38,-0.676,-1.612426,0.000000,0.0,0.000000
10599,2024-03-26,República Tcheca Zbl Feminina,12:30,Slovanka F,Brandys nad Labem F,1.34,3.08,148.5,1.81,1.89,0.0,0.0,0.0,4Sw7Wnoj,0.746269,0.324675,0.552486,0.529101,0.070944,224.118,108.431918,0.483816,1.2,1.643168,1.369306,139.75,3.278,1.794650,0.547483,-43.0,261.422,293.583692,1.123026,0.0,0.000000,NaN,92.40,4.016,4.341662,1.081091,-96.0,65,56,2.15,1.65,317.734,538.368,0.556727,0.030578,NaN,1.20,0.240,1.416667,0.000000,0.0,0.000000,2.11,0.422,4.928910,0.000000,0.0,0.000000
10600,2024-03-26,Lituânia Moteru Lyga Feminina,13:00,Kibirkstis Vilnius F,Panevezys F,1.08,20.00,142.5,1.85,1.85,0.0,0.0,0.0,dAIK1cd2,0.925926,0.050000,0.540541,0.540541,-0.024074,501.262,883.197552,1.761948,1.8,1.643168,0.912871,2080.00,6.542,10.895697,1.665499,37.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,80,46,26.00,10.00,0.000,0.000,1.269304,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10601,2024-03-26,Lituânia Moteru Lyga Feminina,12:30,Siauliai F,Aistes LSU Kaunas F,1.18,4.47,140.5,1.85,1.85,0.0,0.0,0.0,OdKC3JRk,0.847458,0.223714,0.540541,0.540541,0.071171,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,578.562,632.800383,1.093747,0.0,0.000000,NaN,383.94,9.868,10.392712,1.053173,-64.0,52,54,5.19,7.11,0.000,0.000,0.823498,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10602,2024-03-26,Israel Liga Leumit,14:30,Maccabi Rishon,Elitzur Shomron,1.47,2.56,169.5,1.81,1.89,0.0,0.0,0.0,zwa3KRgD,0.680272,0.390625,0.552486,0.529101,0.070897,177.014,70.964530,0.400898,1.2,1.643168,1.369306,235.56,2.114,0.846510,0.400431,-1.0,211.206,98.676495,0.467205,0.6,1.341641,2.236068,89.70,2.474,0.923380,0.373233,-43.0,78,69,3.02,1.30,138.008,252.846,0.382504,0.030578,NaN,0.79,0.158,2.974684,0.000000,0.0,0.000000,2.17,0.434,3.594470,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10673,2024-03-26,Coreia Do Sul Kbl,07:00,KCC Egis,Anyang JungKwanJang,1.49,2.59,177.5,1.85,1.89,0.0,0.0,0.0,8xS4wu4n,0.671141,0.386100,0.540541,0.529101,0.057241,166.196,65.811969,0.395990,1.2,1.643168,1.369306,110.49,1.700,0.515218,0.303070,-1.0,322.670,154.109172,0.477606,1.8,1.643168,0.912871,384.96,3.984,1.909249,0.479229,17.0,87,96,1.27,4.01,198.360,292.522,0.381283,0.015125,NaN,0.34,0.068,7.205882,0.599310,0.7,0.100690,-0.99,-0.198,-8.030303,0.313219,0.1,-0.213219
10674,2024-03-26,China Wcba Feminina,08:30,Sichuan F,Shanxi Xing Rui F,1.03,10.50,158.5,1.83,1.87,0.0,0.0,0.0,46n9faKg,0.970874,0.095238,0.546448,0.534759,0.066112,513.622,910.5076

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
10609,07:00,Geórgia Superliga,Orbi,TSU,2.13,Back Home
10633,15:00,Suécia Superettan,Norrort,KFUM Fryshuset,2.34,Back Home
10655,21:00,Eua Nba,New Orleans Pelicans,Oklahoma City Thunder,2.04,Back Home
10656,08:35,China Cba,Shandong,Beijing Royal Fighters,1.86,Back Home
10657,08:35,China Cba,Shanghai,Shenzhen,1.67,Back Home
